In [1]:
import logging
from strands import Agent
import boto3
from strands.models import BedrockModel
from strands_tools import calculator
import asyncio
from strands_tools import shell
from strand_parallel_pipe import ParallelStrand
import json
from typing import Dict, Any, Union
import sys
import io
from jinja2 import Template
import asyncio

import nest_asyncio
nest_asyncio.apply()

bedrock_model = BedrockModel(
    model_id="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
    temperature=0.3,
)

async def main_single_query(input_text):
    try:
        agent_one = Agent(
            model=bedrock_model,
            callback_handler=None,
            system_prompt=f"""
            당신은 천문학 전문가입니다. {input_text}의 온도에 대해 말해주세요."""
        )
        
        agent_two = Agent(
            model=bedrock_model,
            callback_handler=None,
            system_prompt=f"""
            당신은 천문학 전문가입니다.  {input_text}의 구성하는 성분에 대해 말해주세요."""        
        )
        
        agent_three = Agent(
            model=bedrock_model,
            callback_handler=None,
            system_prompt=f"""
            당신은 천문학 전문가입니다.  {input_text}의 갖고있는 특성에 대해 말해주세요."""   
        )
        
        agent_dict = {
            "temperature": agent_one,
            "compose": agent_two,
            "features": agent_three
        }
        
        parallel_strand = ParallelStrand(agent_dict)
        result = await parallel_strand.invoke("system prompt의 지시를 바탕으로 실행해줘")
        return result
    except Exception as e:
        print(f"Error in main_single_query: {e}")
        import traceback
        traceback.print_exc()
        return None






In [2]:
result = await main_single_query("태양")

In [3]:
result

{'temperature': '안녕하세요! 천문학 전문가로서 태양의 온도에 대해 자세히 설명드리겠습니다.\n\n## 태양의 온도 구조\n\n태양은 층별로 다른 온도를 가지고 있습니다:\n\n### 1. 핵 (Core)\n- **온도**: 약 1,500만°C (1.5 × 10⁷ K)\n- 핵융합 반응이 일어나는 곳으로 가장 뜨거운 부분입니다\n\n### 2. 복사층 (Radiative Zone)\n- **온도**: 약 200만°C ~ 700만°C\n- 에너지가 복사를 통해 천천히 바깥쪽으로 전달됩니다\n\n### 3. 대류층 (Convection Zone)\n- **온도**: 약 200만°C ~ 5,800°C\n- 대류 현상을 통해 에너지가 전달됩니다\n\n### 4. 광구 (Photosphere) - 태양 표면\n- **온도**: 약 5,778K (5,505°C)\n- 우리가 보는 태양의 표면으로, 빛을 방출하는 층입니다\n\n### 5. 채층 (Chromosphere)\n- **온도**: 약 4,000°C ~ 25,000°C\n- 표면에서 멀어질수록 온도가 상승합니다\n\n### 6. 코로나 (Corona)\n- **온도**: 100만°C ~ 300만°C\n- 태양의 가장 바깥층으로 매우 뜨겁지만 밀도가 낮습니다\n\n이러한 온도 분포는 태양의 구조와 에너지 전달 메커니즘을 이해하는 데 매우 중요합니다.',
 'compose': "태양의 구성 성분에 대해 설명드리겠습니다.\n\n## 태양의 주요 구성 성분\n\n### **수소 (Hydrogen) - 약 73%**\n- 태양 질량의 대부분을 차지하는 가장 주요한 성분\n- 핵융합 반응의 연료 역할\n- 태양 중심부에서 헬륨으로 변환되며 에너지 생성\n\n### **헬륨 (Helium) - 약 25%**\n- 두 번째로 많은 구성 성분\n- 수소 핵융합의 결과물\n- 태양의 진화 과정에서 점진적으로 증가\n\n### **기타 중원소 - 약 2%**\n천문학에서 '금속'이라 부르는 수소와 헬륨보다 무거운 원소들:

In [4]:
def convert(template, result, role=""):
    return str(template.render(role=role, **result))

In [5]:
system_prompt = Template("""
당신은 천문학 전문가입니다.

{% if temperature %}
이건 대상의 온도에 관한 정보입니다:
{{ temperature }}...
{% endif %}

{% if compose %}
이건 대상의 구성에 관한 정보입니다:
{{ compose }}...
{% endif %}

{% if features %}
이건 대상의 특징에 관한 정보입니다:
{{ features }}...
{% endif %}

위의 정보를 바탕으로 다각도로 대상을 분석해주세요
""".strip()).render( **result)

In [6]:
system_prompt

"당신은 천문학 전문가입니다.\n\n\n이건 대상의 온도에 관한 정보입니다:\n안녕하세요! 천문학 전문가로서 태양의 온도에 대해 자세히 설명드리겠습니다.\n\n## 태양의 온도 구조\n\n태양은 층별로 다른 온도를 가지고 있습니다:\n\n### 1. 핵 (Core)\n- **온도**: 약 1,500만°C (1.5 × 10⁷ K)\n- 핵융합 반응이 일어나는 곳으로 가장 뜨거운 부분입니다\n\n### 2. 복사층 (Radiative Zone)\n- **온도**: 약 200만°C ~ 700만°C\n- 에너지가 복사를 통해 천천히 바깥쪽으로 전달됩니다\n\n### 3. 대류층 (Convection Zone)\n- **온도**: 약 200만°C ~ 5,800°C\n- 대류 현상을 통해 에너지가 전달됩니다\n\n### 4. 광구 (Photosphere) - 태양 표면\n- **온도**: 약 5,778K (5,505°C)\n- 우리가 보는 태양의 표면으로, 빛을 방출하는 층입니다\n\n### 5. 채층 (Chromosphere)\n- **온도**: 약 4,000°C ~ 25,000°C\n- 표면에서 멀어질수록 온도가 상승합니다\n\n### 6. 코로나 (Corona)\n- **온도**: 100만°C ~ 300만°C\n- 태양의 가장 바깥층으로 매우 뜨겁지만 밀도가 낮습니다\n\n이러한 온도 분포는 태양의 구조와 에너지 전달 메커니즘을 이해하는 데 매우 중요합니다....\n\n\n\n이건 대상의 구성에 관한 정보입니다:\n태양의 구성 성분에 대해 설명드리겠습니다.\n\n## 태양의 주요 구성 성분\n\n### **수소 (Hydrogen) - 약 73%**\n- 태양 질량의 대부분을 차지하는 가장 주요한 성분\n- 핵융합 반응의 연료 역할\n- 태양 중심부에서 헬륨으로 변환되며 에너지 생성\n\n### **헬륨 (Helium) - 약 25%**\n- 두 번째로 많은 구성 성분\n- 수소 핵융합의 결과물\n- 태양의 진화 과정에서 점진적으로 증가\n\n### **기타 중원소 

In [7]:
agent_summary = Agent(
    model=bedrock_model,
    system_prompt=system_prompt,
    callback_handler=None
)

In [8]:
from IPython.display import display, HTML, clear_output
import asyncio

async def run_stream():
    output = ""
    async for event in agent_summary.stream_async("system Prompt의 지시에 따라 사용해 주세요"):
        if "data" in event:
            output += event["data"]
            clear_output(wait=True)
            display(HTML(f"<pre>{output}</pre>"))
            await asyncio.sleep(0.01)  # 화면 업데이트 시간
    print()  # 줄바꿈


In [9]:
await run_stream()
